# Passo 0: Setup

## Prepare environment

In [ ]:
!pip install scikit-learn==1.2.2

## Criando as constantes

In [3]:
class Constants:
    MIN_AGE = 18
    MAX_AGE = 120
    MIN_IMC = 0.0
    MAX_IMC = 50.0
    MIN_CHILDREN = 0
    MAX_CHILDREN = 10
    SEXO_MAP = {
        "masculino": 0,
        "feminino": 1
    }
    FUMANTE_MAP = {
        "nao": 0,
        "sim": 1
    }
    REGIAO_MAP = {
        "norte": 0,
        "nordeste": 1,
        "centro oeste": 2,
        "sul": 3,
        "sudeste": 4
    }

## Criando a classe Util

In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

class Util:

    @staticmethod
    def calculate_rmse(mse):
        return np.sqrt(mse)


    @staticmethod
    def calculate_mape(labels, predictions):
        errors = np.abs(labels - predictions)
        relative_errors = errors / np.abs(labels)
        mape = np.mean(relative_errors) * 100
        return mape

    @staticmethod
    def evaluate(model, x_test, y_test):
        y_pred = model.predict(x_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = Util.calculate_rmse(mse)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        mape =  Util.calculate_mape(y_test, y_pred)
        print(f'Mean squared error: {mse}')
        print(f'Sqrt Mean squared error: {rmse}')
        print(f'R2 score: {r2}')
        print(f'Mean absolute error: {mae}')
        print(f'MAPE: {mape:.2f}%')

## Criando a classe SaveManager

In [5]:
import joblib

class SaveModelManager:

    @staticmethod
    def save_model(model, filename):
        joblib.dump(model, f'model/{filename}')
        print(f'Saved model: {filename}')

    @staticmethod
    def load_model(filename):
        model = joblib.load(filename)
        return model

## Criando as classes TrainModel

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
import matplotlib.pyplot as plt

class TrainModelBase:

    def __init__(self, p_name):
        self.name_model = p_name
        self.model = None
        self.x_train = None
        self.x_test = None
        self.y_train = None
        self.y_test = None
        self.random_state = 42


    def train(self, X, y, test_size):
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(X, y, test_size=test_size, random_state=self.random_state)
        print(len(self.x_train), "training +", len(self.x_test), "tests")
        print(len(self.y_train), "training +", len(self.y_test), "tests")
        return self.x_train, self.x_test, self.y_train, self.y_test


    def get_model(self):
        return self.model


    def evaluate(self):
        Util.evaluate(self.model, self.x_test, self.y_test)


    def evaluate_pred(self):
        y_pred = self.model.predict(self.x_test)
        plt.scatter(self.y_test, y_pred)
        plt.xlabel("Valores reais")
        plt.ylabel("Previsões")
        plt.title("Comparando")
        plt.show()


    def save(self):
        filename = self.name_model + '.pkl'
        SaveModelManager.save_model(self.model, filename)


class TrainModelLinearRegression(TrainModelBase):

    def generate_model(self):
        self.model = LinearRegression()
        self.model.fit(self.x_train, self.y_train)
        return self.model


class TrainModelRidgeRegression(TrainModelBase):

    def generate_model(self):
        self.model = Ridge(alpha=1.0, random_state=self.random_state)
        self.model.fit(self.x_train, self.y_train)
        return self.model


class TrainModelLassoRegression(TrainModelBase):

    def generate_model(self):
        self.model = Lasso(alpha=0.1, random_state=self.random_state)
        self.model.fit(self.x_train, self.y_train)
        return self.model


class TrainModelElasticNetRegression(TrainModelBase):

    def generate_model(self):
        self.model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=self.random_state)
        self.model.fit(self.x_train, self.y_train)
        return self.model


class TrainModelGradientBoostingRegressor(TrainModelBase):

    def generate_model(self):
        self.model = GradientBoostingRegressor(random_state=self.random_state)
        self.model.fit(self.x_train, self.y_train)
        return self.model


class TrainModelSupportVectorRegression(TrainModelBase):

    def generate_model(self):
        self.model = SVR(kernel='linear')
        self.model.fit(self.x_train, self.y_train)
        return self.model


class TrainModelRandomForestRegressor(TrainModelBase):

    def generate_model(self):
        self.model = RandomForestRegressor(n_estimators=100, random_state=self.random_state)
        self.model.fit(self.x_train, self.y_train)
        return self.model


## Criando o app UI

# App UI

In [12]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import os

def build_and_start_app():
  model_list = os.listdir('model')
  model_list = [file for file in model_list if file.endswith('.pkl')]

  title = widgets.HTML(
      value="<h1 style='text-align: center;'>Previsão de custo de plano de saúde",
      layout=widgets.Layout(margin='10px 0px')
  )
  model_widget = widgets.Dropdown(options=model_list, description='Modelo:',layout=widgets.Layout(width='75%'))
  idade_widget = widgets.IntSlider(value=25, min=Constants.MIN_AGE, max=Constants.MAX_AGE, description='Idade:',layout=widgets.Layout(width='75%'))
  sexo_widget = widgets.Dropdown(options=Constants.SEXO_MAP.items(), description='Sexo:',layout=widgets.Layout(width='75%'))
  imc_widget = widgets.FloatSlider(value=25.0, min=Constants.MIN_IMC, max=Constants.MAX_IMC, description='IMC:',layout=widgets.Layout(width='75%'))
  criancas_widget = widgets.Dropdown(options=list(range(10)), description='Nº crianças:',layout=widgets.Layout(width='75%'))
  fumante_widget = widgets.Dropdown(options=Constants.FUMANTE_MAP.items(), description='Fumante:',layout=widgets.Layout(width='75%'))
  regiao_widget = widgets.Dropdown(options=Constants.REGIAO_MAP.items(), description='Região:',layout=widgets.Layout(width='75%'))
  submit_button = widgets.Button(description="Prever custo", button_style='success', icon='check',layout=widgets.Layout(width='75%'))

  output_result_label = widgets.HTML(
      value="<p style='font-size: 16px; color: #B6B6B6;'>Valor estimado: aguardando execução...</p>",
      layout=widgets.Layout(margin='10px 0px')
  )

  def prever_custo(b):
      if not model_list:
        output_result_label.value = f'<p style="font-size: 16px; color: #B6B6B6;">Você ainda não tem nenhum modelo gerado.<br>Cheque a seção <b>Comece aqui</b> para saber mais.</p>'
        return
      model_saved = SaveModelManager.load_model(f'model/{model_widget.value}')
      data = pd.DataFrame({
          'idade': [idade_widget.value],
          'imc': [imc_widget.value],
          'criancas': [criancas_widget.value],
          'sexo_1': [bool(sexo_widget.value)],
          'fumante_1': [bool(fumante_widget.value)],
          'regiao_1': [regiao_widget.value == 1],
          'regiao_2': [regiao_widget.value == 2],
          'regiao_3': [regiao_widget.value == 3],
          'regiao_4': [regiao_widget.value == 4]
      })
      previsao = model_saved.predict(data)
      output_result_label.value = f'<p style="font-size: 16px; color: #B6B6B6;">Modelo executado: {model_widget.value}<br>Valor estimado: R$ {previsao[0]:.2f}</p>'

  submit_button.on_click(prever_custo)

  form_items = [
      title,
      model_widget, idade_widget, sexo_widget, imc_widget, criancas_widget, fumante_widget, regiao_widget, submit_button,
      output_result_label
  ]

  form = widgets.VBox(form_items, layout=widgets.Layout(
      width='50%',
      border='solid 1px lightgray',
      padding='10px',
      margin='10px'
  ))

  display(form)

## Start App

In [13]:
build_and_start_app()